In [ ]:
from neuroglancer_annotation_ui.statebuilder import StateBuilder, ChainedStateBuilder
from annotationframeworkclient.infoservice import InfoServiceClient
import pandas as pd
import numpy as np

neuroglancer_prefix = 'https://graphene-v0-dot-neuromancer-seung-import.appspot.com/' 

##### The following code generated the example dataframes
```
dl = AnalysisDataLink(dataset_name=dataset_name,
                      sqlalchemy_database_uri=sql_database_uri_base,
                      materialization_version=data_version,
                      verbose=False)

soma_df = dl.query_cell_types(soma_table)
pre_syn_df = dl.query_synapses(synapse_table, pre_ids=[648518346349509367])
post_syn_df = dl.query_synapses(synapse_table, post_ids=[648518346349509367])

soma_df.to_hdf('example_dataframes.h5', 'soma_df')
pre_syn_df.to_hdf('example_dataframes.h5', 'pre_syn_df')
post_syn_df.to_hdf('example_dataframes.h5', 'post_syn_df')
```

In [ ]:
soma_df = pd.read_hdf('example_dataframes.h5', 'soma_df')
pre_syn_df = pd.read_hdf('example_dataframes.h5', 'pre_syn_df')
post_syn_df = pd.read_hdf('example_dataframes.h5', 'post_syn_df')

## State Builder and Materialized Data Queries

### Basic Options
The StateBuilder class takes a set of rules for reading the different columns of a dataframe and converting them to Neuroglancer states. The most basic form has an image and segmentation layer, with some column to use to source point annotations. For ease, it can take a dataset name to look up image and segmentation information.

In [ ]:
dataset_name = 'pinky100'
annotation_map = {'somata': ['pt_position']}

point_builder = StateBuilder(dataset_name=dataset_name,
                             point_annotations=annotation_map)

point_builder.render_state(soma_df, return_as='html', url_prefix=neuroglancer_prefix)

The same information can be returned as JSON, as a url string, or as a neuroglancer viewer.

Image and segmentation layers can also be specified manually, and so can the url prefix.

In [ ]:
dataset_name = 'pinky100'
info = InfoServiceClient(dataset_name=dataset_name)
img_dict = {'img': info.image_source(format_for='neuroglancer')}
seg_dict = {'seg': info.pychunkedgraph_viewer_source(format_for='neuroglancer')}

point_builder = StateBuilder(image_sources=img_dict,
                             seg_sources=seg_dict,
                             point_annotations=annotation_map)

point_builder.render_state(soma_df, return_as='html', url_prefix=neuroglancer_prefix)

### Soma Points by Cell Type
In order to make this more responsive to data, we pivot the dataframe to make cell-type specific layers with different colors. If we wanted to not anticipate the results, we could always generate the annotation map from the dataframe itself.

In [ ]:

annotation_map = {'e': ['e'],
                  'i': ['i'], 
                  'g': ['g']} 
annotation_layer_colors = {'e': '#ffffff', 'i':'#6dc8d0', 'g':'#011c7f'}
cell_type_builder = StateBuilder(dataset_name=dataset_name,
                                 seg_sources=seg_dict,
                                 annotation_layer_colors=annotation_layer_colors,
                                 point_annotations=annotation_map)

dfp = soma_df.pivot(values='pt_position', columns='cell_type')
cell_type_builder.render_state(dfp, return_as='html', url_prefix=neuroglancer_prefix)

You can also collect points from multiple columns into the same layer.

In [ ]:
annotation_map = {'neurons': ['e', 'i'],
                  'glia': ['g']}
annotation_layer_colors = {'neurons': '#fffff', 'glia':'#5dc7d0'}

cell_type_builder = StateBuilder(dataset_name=dataset_name,
                                 point_annotations=annotation_map,
                                 annotation_layer_colors=annotation_layer_colors)

dfp = soma_df.pivot(values='pt_position', columns='cell_type')
cell_type_builder.render_state(dfp, return_as='html', url_prefix=neuroglancer_prefix)

## Additional types of rendering

You can also select object ids based on a column of the dataframe. In addition, line and sphere annotations can be created in much the same way as the point annotation, but with a list of tuples.

In [ ]:
line_annotations = {'synapses': [['ctr_pt_position', 'post_pt_position']]}
color_map = {'synapses': '#ff1200'}
selection_map = {'seg': ['pre_pt_root_id']}    # These will be selected by column

presyn_builder = StateBuilder(dataset_name=dataset_name,
                              annotation_layer_colors=color_map,
                              selected_ids=selection_map,
                              line_annotations=line_annotations,
                              )

presyn_builder.render_state( pre_syn_df, return_as='html', url_prefix=neuroglancer_prefix)

In [ ]:
fixed_selection = {'seg': [648518346349537978]}    # These will always be selected
presyn_builder = StateBuilder(dataset_name=dataset_name, 
                              annotation_layer_colors=color_map,
                              selected_ids=selection_map,
                              line_annotations=line_annotations,
                              fixed_selection=fixed_selection,
                              )

presyn_builder.render_state( pre_syn_df, return_as='html', url_prefix=neuroglancer_prefix)

### Using a Base State

Instead of giving all components, one can also give an initial neuroglancer state onto which the dataframe annotations and selections will add.

In [ ]:
from annotationframeworkclient import jsonservice
stateclient = jsonservice.JSONService()
base_state = stateclient.get_state_json(state_id=5721488686448640)

In [ ]:
line_annotations = {'synapses': [['ctr_pt_position', 'post_pt_position']]}
color_map = {'synapses': '#ff1200'}
selection_map = {'seg': ['pre_pt_root_id']}    # These will be selected by column

presyn_builder = StateBuilder(base_state=base_state,
                              annotation_layer_colors=color_map,
                              selected_ids=selection_map,
                              line_annotations=line_annotations,
                              )

presyn_builder.render_state( pre_syn_df, return_as='html', url_prefix=neuroglancer_prefix)

### Chaining multiple dataframes and builders
If we want to use multiple dataframes and multiple statebuilders, we can chain them together, passing the output state of one to the base state of the next. As a convenience, this is wrapped up into the concept of a ChainedStateBuilder. After making each individual StateBuilder, you can link them together into a ChainedStateBuilder that will take collections of dataframes together and process them in the same order. One use case for this would be pre- and postsynaptic points. Note that because most of the state has been set up by the first step, the second builder can just add annotations.

In [ ]:
pre_annotation_map = {'pre': ['ctr_pt_position']}
color_map = {'pre': '#FF1200', 'post': '#03d7ff'}
pre_selection = {'seg': ['pre_pt_root_id']}

presyn_builder = StateBuilder(dataset_name=dataset_name,
                              selected_ids=pre_selection,
                              point_annotations=pre_annotation_map,
                              annotation_layer_colors=color_map,
                              )

post_annotation_map = {'post': ['ctr_pt_position']}
postsyn_builder = StateBuilder(point_annotations=post_annotation_map)

chained_builder = ChainedStateBuilder((presyn_builder, postsyn_builder))
chained_builder.render_state((pre_syn_df, post_syn_df), return_as='html', url_prefix=neuroglancer_prefix)

### Building states from arrays
The StateBuilder class is designed to handle schematized dataframes, like those we get back from queries and analysis pipelines. However, if you're doing analysis and you have specific points / ids that you want to highlight, you can directly build a state without explicitly going through the dataframe stage.

In [ ]:
from neuroglancer_annotation_ui.statebuilder import build_state_direct

In [ ]:
example_pts_pre = np.vstack(pre_syn_df['pre_pt_position'].values)
example_pts_post = np.vstack(post_syn_df['post_pt_position'].values)
example_ids = [648518346349509367]

A simple example, using direct numpy arrays and lists.

In [ ]:
build_state_direct(dataset_name='pinky100', point_annotations=example_pts_pre, selected_ids=example_ids, return_as='html', render_kws={'url_prefix': neuroglancer_prefix})

A more complex example, using multiple layers and kwargs.

In [ ]:
build_state_direct(dataset_name='pinky100', point_annotations={'pre':example_pts_pre, 'post':example_pts_post}, selected_ids=example_ids, return_as='html',
                   annotation_layer_colors= {'pre': '#ff2300', 'post':'#2300ff'},
                   render_kws={'url_prefix':neuroglancer_prefix})